In [ ]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import re
import base64
import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

from databasecrud import DatabaseCRUD


###########################
# Data Manipulation / Model
###########################
username = "aacuser" # database username
password = "aacPassword" # database password
port = "37346" # MongoDB port
database = "AAC" # database to connect to
collection = "animals" # collection within database

shelter = DatabaseCRUD(username, password, port, database, collection) # instance of CRUD Python module 

df = pd.DataFrame.from_records(shelter.read({}))

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

image_filename = 'grazioso_salvare_logo.png' # logo image
encoded_image = base64.b64encode(open(image_filename, 'rb').read()) # encode image

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    # Logo and header
    html.Div([
              html.A([
                      html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), # logo
                               width="150", # image width
                               height="150") # image height
                     ], href="https://www.snhu.edu/"), # click image to link to SNHU homepage
              html.H1('Jordan Ballard CS-340 Dashboard') # header
        ], 
        style={'display':'flex', 'align-items':'center'}), # place logo and header on same line and align
    # Line
    html.Hr(),
    # Radio Items
    html.Div(dcc.RadioItems(id='filter-type',
                            options=[
                                {'label':'Water Rescue', 'value': 'water'}, # water rescue filter
                                {'label':'Wilderness Rescue', 'value': 'wilderness'}, # wilderness rescue filter
                                {'label': 'Disaster Rescue', 'value': 'disaster'}, # disaster rescue filter
                                {'label': 'Reset', 'value': 'reset'} # reset filters
                            ], 
                            value='choice')),
    # Line
    html.Hr(),
    # Table
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable = False, # non-editable table
        filter_action = "native", # display filtering UI
        sort_action = "none", # no sorting UI
        sort_mode = "multi", # multi column sort
        column_selectable = False, # cannot select columns
        row_selectable = 'single', # can select rows
        row_deletable = False, # cannot delete rows
        selected_columns = [], # list of currently selected columns
        selected_rows = [0], # list of currently selected rows, use 0 to automatically start at first row
        page_action = "native", # separate rows into pages
        page_current = 0, # starting page
        page_size = 10, # rows per page
    ),
    # Break
    html.Br(),
    # Line
    html.Hr(),
    #This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################

### This callback controls the radio items ###
@app.callback(
    [Output('datatable-id', 'data'),
    Output('datatable-id', 'columns')],
    [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    # if water rescue button is selected, perform this query
    # use Python regex to match dogs of mixed breed
    if (filter_type == 'water'):
        df = pd.DataFrame.from_records(shelter.read({"animal_type": "Dog",
                                                     "breed":{"$in":[re.compile("Labrador Retriever.*"),
                                                                     re.compile("Chesa Bay Retr.*"),
                                                                     re.compile("Newfoundland.*")]},
                                                     "sex_upon_outcome":"Intact Female",
                                                     "age_upon_outcome_in_weeks":{"$gte":26,
                                                                                  "$lte":156}
                                             }))
    # if wilderness button is selected, perform this query
    elif (filter_type == 'wilderness'):
        df = pd.DataFrame.from_records(shelter.read({"animal_type": "Dog",
                                                     "breed":{"$in":[re.compile("German Shepherd.*"),
                                                                     re.compile("Alaskan Malamute.*"),
                                                                     re.compile("Old English Sheepdog.*"),
                                                                     re.compile("Siberian Husky.*"),
                                                                     re.compile("Rottweiler.*")]},
                                                     "sex_upon_outcome":"Intact Male",
                                                     "age_upon_outcome_in_weeks":{"$gte":26,
                                                                                  "$lte":156}
                                             }))
    # if disaster button is selected, perform this query
    elif (filter_type == 'disaster'):
        df = pd.DataFrame.from_records(shelter.read({"animal_type": "Dog",
                                                     "breed":{"$in":[re.compile("Doberman Pinsch.*"),
                                                                     re.compile("German Shepherd.*"),
                                                                     re.compile("Golden Retriever.*"),
                                                                     re.compile("Bloodhound.*"),
                                                                     re.compile("Rottweiler.*")]},
                                                     "sex_upon_outcome":"Intact Male",
                                                     "age_upon_outcome_in_weeks":{"$gte":20,
                                                                                  "$lte":300}
                                             }))
    # if reset filter is selected, reset table to show all animals
    elif (filter_type == 'reset'):
        df = pd.DataFrame.from_records(shelter.read({}))
    
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data=df.to_dict('records')
        
    return (data,columns)


### This callback builds the pie chart ###
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "data")])
def update_graphs(viewData):
    dff = pd.DataFrame.from_dict(viewData) # data to view
    fig = px.pie(dff, names="breed") # create pie graph with animal data and show breed distribution
    
    return dcc.Graph(figure=fig)


### This callback builds the map ###
@app.callback(
    Output('map-id', "children"), # output children to map-id
    [Input('datatable-id', "data"), # input derived_viewport_data from datatable-id
     Input('datatable-id', "selected_rows") # input selected_rows from datatable-id
    ])
def update_map(viewData, selected_rows):
    dff = pd.DataFrame.from_dict(viewData) # data to view
    row = selected_rows[0] # selected row index
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, # width and height of map
               center=[30.75,-97.48], # Austin TX is at [30.75,-97.48]
               zoom=10, 
               children=[
                   dl.TileLayer(id="base-layer-id"),
                   # Marker with tool tip and popup
                   # Position is lat and long for selected animal
                   dl.Marker(position=[dff.iloc[row,13], dff.iloc[row,14]] , children=[
                   # tooltip info on hover
                   dl.Tooltip(dff.iloc[row, 4]), # breed of selected animal
                   # popup info on click
                   dl.Popup([
                       html.H1("Animal Name"),
                       html.P(dff.iloc[row,9]) # name of selected animal
                   ])
               ])
           ])
       ]


app